<a href="https://colab.research.google.com/github/ZijinDai/SOSC314-PROJECT/blob/main/%23ICE_shooting_data_collection_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt-dlp

In [ ]:
!yt-dlp "https://www.youtube.com/hashtag/iceshooting" \
    --skip-download \
    --write-info-json \
    --write-comments \
    --write-auto-subs \
    --sub-lang "en.*" \
    --output "/content/hashtag_data/%(title)s.%(ext)s"

In [ ]:
!pip install webvtt-py

In [ ]:
import json
import glob
import os
import pandas as pd
import webvtt

def clean_vtt(vtt_path):
    if not os.path.exists(vtt_path): return ""
    try:
        vtt = webvtt.read(vtt_path)
        lines = []
        for line in vtt: lines.extend(line.text.strip().splitlines())
        res, prev = [], None
        for l in lines:
            if l != prev: res.append(l); prev = l
        return "\n".join(res)
    except: return ""

video_list = []
comment_list = []

data_folder = "/content/hashtag_data"
json_files = glob.glob(os.path.join(data_folder, "*.info.json"))

for json_path in json_files:
    with open(json_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)

    v_id = meta.get('id') # Unique video ID

    # 1. Video metadata and subtitles
    vtt_files = glob.glob(json_path.replace(".info.json", "*.vtt"))
    transcript = clean_vtt(vtt_files[0]) if vtt_files else ""

    video_list.append({
        "video_id": v_id,
        "title": meta.get('title'),
        "upload_date": meta.get('upload_date'),
        "channel": meta.get('channel'),
        "view_count": meta.get('view_count'),
        "description": meta.get('description'),
        "transcript": transcript
    })

    # 2. Comments
    comments = meta.get('comments', [])
    for c in comments:
        # 1. Extract the original timestamp
        raw_ts = c.get('timestamp')

        # 2. Convert timestamp to a numeric format, and then to a date format
        try:
            if raw_ts:
                readable_ts = pd.to_datetime(pd.to_numeric(raw_ts), unit='s')
            else:
                readable_ts = None
        except Exception:
            readable_ts = None

        comment_list.append({
            "video_id": v_id,
            "author": c.get('author'),
            "comment_text": c.get('text'),
            "timestamp": readable_ts,
            "like_count": c.get('like_count')
        })

# --- Save as double sheets using Pandas' ExcelWriter ---
output_file = "ICE_Research_Relational_Data.xlsx"
with pd.ExcelWriter(output_file) as writer:
    pd.DataFrame(video_list).to_excel(writer, sheet_name='Video_Summary', index=False)
    pd.DataFrame(comment_list).to_excel(writer, sheet_name='Comment_Details', index=False)

print(f"Double sheets completed: {output_file}")